# **Flight Delay Prediction using Scikit-Learn Pipeline**

## **Overview**
This project demonstrates how to build a **machine learning pipeline** using scikit-learn to predict flight delays. The pipeline integrates data preprocessing with model training, ensuring efficient handling of both numerical and categorical data.

---

## **Objectives**
- Preprocess numerical and categorical data using `ColumnTransformer`.
- Automate the machine learning workflow using `Pipeline`.
- Train a **Random Forest Classifier** to predict flight delays.
- Optimize the model using **GridSearchCV** for hyperparameter tuning.

---

## **Data Overview**
- **Dataset**: Contains flight details such as:
  - **Year**, **Month**, **Day**
  - **Airline code**, **Origin airport code**, **Destination airport code**
  - **Departure delay** (target: delayed or not)

- **Target Variable**:  
  - `1` if the flight was delayed  
  - `0` if the flight was on time

---

## **Steps Involved**

### 1. **Data Loading and Exploration**
- Load the flight dataset and inspect its structure and missing values.

### 2. **Feature Engineering**
- **Numerical Features**:
  - `YEAR`, `MONTH`, `DAY`
- **Categorical Features**:
  - `AIRLINE__CODE`, `ORIGIN_AIRPORT_CODE`, `DESTINATION_AIRPORT_CODE`

### 3. **Preprocessing with `ColumnTransformer`**
- **Numerical Data**:
  - Impute missing values with the **mean**.
  - Standardize values using **`StandardScaler`**.
  
- **Categorical Data**:
  - Impute missing values with `'missing'`.
  - Encode using **`OneHotEncoder`**.

### 4. **Pipeline Setup**
- Use a **scikit-learn Pipeline** to link preprocessing and model training.
- Integrate a **Random Forest Classifier** within the pipeline.

### 5. **Model Training and Evaluation**
- Split the data into **train (70%)** and **test (30%)** sets.
- Evaluate the model using a **classification report** with metrics like:
  - **Precision**, **Recall**, **F1-score**

### 6. **Hyperparameter Tuning with GridSearchCV**
- Tune hyperparameters of the Random Forest model:
  - Number of estimators (`n_estimators`)
  - Maximum tree depth (`max_depth`)

### 7. **Model Persistence**
- Save the trained model using **`joblib`** for later use.

---

## **Technologies Used**
- **Python**: Programming language
- **Pandas**: Data manipulation and cleaning
- **Scikit-Learn**: Machine learning, preprocessing, and model evaluation
- **Joblib**: Model persistence
- **Jupyter Notebook**: Interactive development environment

---

## **Expected Output**
- A **trained Random Forest model** to predict flight delays.
- **Performance metrics** (accuracy, precision, recall) from the classification report.
- A **saved model** (`flight_delay_classifier.pkl`) for deployment.

---

## **Conclusion**
This project demonstrates how to create an automated **machine learning workflow** using scikit-learn’s `Pipeline` and `ColumnTransformer`. The streamlined preprocessing ensures consistency during both training and testing. With **hyperparameter tuning**, the model's performance is further optimized, making it reliable for real-world flight delay predictions.


# Upgrade pip and install all required packages

In [ ]:
!pip install --upgrade pip

# Install Snowflake connectors, pandas integration, and essential libraries
!pip install "snowflake-connector-python[pandas]" \
             snowflake-snowpark-python==1.9.0 \
             numpy pandas matplotlib scikit-learn xgboost seaborn \
             python-dateutil tqdm holidays faker

# Ensure Snowpark Python is up-to-date
!pip install --upgrade -q snowflake-snowpark-python==1.9.0

# Fix potential urllib3 version conflicts
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

# Additional installations for your project
!pip install fosforml==1.1.6
!pip install python-scipy
!pip install basemap


# Importing necessary libraries and settings

In [1]:

# Standard libraries for date and warnings
import datetime
import warnings

# Scientific and Data Manipulation Libraries
import scipy
import pandas as pd
import numpy as np

# Data Visualization Libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb

# Sklearn Modules for Data Preprocessing, Modeling, and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder  # Encoding categorical variables
from sklearn.preprocessing import StandardScaler  # Scaling numerical data
from sklearn.tree import DecisionTreeClassifier  # Decision Tree model
from sklearn.metrics import roc_auc_score, classification_report  # Evaluation metrics

# Configuring display options and warning filters
pd.options.display.max_columns = 50
warnings.filterwarnings("ignore")

# Custom FosforML package for Snowflake session and model registration
from fosforml.model_manager.snowflakesession import get_session
from fosforml import register_model
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer


In [2]:
# Set Matplotlib's default font family to 'DeJavu Serif' to ensure a consistent font style across plots
plt.rcParams['font.family'] = 'DeJavu Serif'

# Establishing a Snowflake session


In [3]:
my_session = get_session()

# Defining the table name to fetch data from
# table_name = 'FLIGHTS'  # Initial option for table
table_name = 'FLIGHTS_FULL'  # Final table to be used

# Querying the data from the specified Snowflake table
sf_df = my_session.sql("SELECT * FROM {}".format(table_name))

# Converting the Snowflake DataFrame to a pandas DataFrame for local processing
df = sf_df.to_pandas()

df

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLY_DATE,AIRLINE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE
0,2024,12,17,4,MQ,3604,N665MQ,FSD,ORD,548,542.0,-6.0,12.0,554.0,111.0,97.0,77.0,462,711.0,8.0,739,719.0,-20.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,American Eagle Airlines Inc.,Sioux Falls Regional Airport,Sioux Falls,SD,USA,43.58135,-96.74170,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446
1,2024,12,17,4,DL,1799,N603AT,CHA,ATL,549,552.0,3.0,15.0,607.0,56.0,48.0,25.0,106,632.0,8.0,645,640.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Chattanooga Metropolitan Airport (Lovell Field),Chattanooga,TN,USA,35.03527,-85.20379,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
2,2024,12,17,4,EV,4412,N14180,MCI,IAH,550,544.0,-6.0,10.0,554.0,133.0,121.0,105.0,643,739.0,6.0,803,745.0,-18.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Atlantic Southeast Airlines,Kansas City International Airport,Kansas City,MO,USA,39.29761,-94.71391,George Bush Intercontinental Airport,Houston,TX,USA,29.98047,-95.33972
3,2024,12,17,4,EV,4309,N26549,PVD,ORD,550,549.0,-1.0,18.0,607.0,176.0,196.0,153.0,849,740.0,25.0,746,805.0,19.0,0,0,None,19.0,0.0,0.0,0.0,0.0,2024-12-17,Atlantic Southeast Airlines,Theodore Francis Green State Airport,Providence,RI,USA,41.72400,-71.42822,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446
4,2024,12,17,4,EV,5143,N146PQ,ORF,DTW,550,545.0,-5.0,18.0,603.0,124.0,128.0,95.0,529,738.0,15.0,754,753.0,-1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Atlantic Southeast Airlines,Norfolk International Airport,Norfolk,VA,USA,36.89461,-76.20122,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2024,12,17,4,OO,6224,N960SW,RDM,DEN,546,559.0,13.0,30.0,629.0,147.0,150.0,114.0,898,923.0,6.0,913,929.0,16.0,0,0,None,3.0,0.0,0.0,0.0,13.0,2024-12-17,Skywest Airlines Inc.,Redmond Municipal Airport (Roberts Field),Redmond,OR,USA,44.25407,-121.14996,Denver International Airport,Denver,CO,USA,39.85841,-104.66700
5819075,2024,12,17,4,UA,1026,N801UA,DCA,IAH,546,544.0,-2.0,9.0,553.0,215.0,206.0,192.0,1208,805.0,5.0,821,810.0,-11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,United Air Lines Inc.,Ronald Reagan Washington National Airport,Arlington,VA,USA,38.85208,-77.03772,George Bush Intercontinental Airport,Houston,TX,USA,29.98047,-95.33972
5819076,2024,12,17,4,NK,224,N633NK,ORD,LGA,546,539.0,-7.0,13.0,552.0,129.0,115.0,91.0,733,823.0,11.0,855,834.0,-21.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Spirit Air Lines,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446,LaGuardia Airport (Marine Air Terminal),New York,NY,USA,40.77724,-73.87261
5819077,2024,12,17,4,UA,1781,N14731,LGA,IAH,547,601.0,14.0,9.0,610.0,249.0,244.0,229.0,1416,859.0,6.0,856,905.0,9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,United Air Lines Inc.,LaGuardia Airport (Marine Air Terminal),New York,NY,USA,40.77724,-73.87261,George Bush Intercontinental Airport,Houston,TX,USA,29.98047,-95.33972


# Filtering data for specific airlines

In [4]:
# Defining the list of airlines to include in the filtered DataFrame
options = ['Southwest Airlines Co.', 'Delta Air Lines Inc.']

# Selecting rows where the 'AIRLINE' column matches one of the specified airlines
flights = df.loc[df['AIRLINE'].isin(options)]
flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLY_DATE,AIRLINE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE
1,2024,12,17,4,DL,1799,N603AT,CHA,ATL,549,552.0,3.0,15.0,607.0,56.0,48.0,25.0,106,632.0,8.0,645,640.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Chattanooga Metropolitan Airport (Lovell Field),Chattanooga,TN,USA,35.03527,-85.20379,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
6,2024,12,17,4,DL,1480,N319US,BDL,DTW,550,549.0,-1.0,20.0,609.0,120.0,129.0,104.0,549,753.0,5.0,750,758.0,8.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Bradley International Airport,Windsor Locks,CT,USA,41.93887,-72.68323,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
7,2024,12,17,4,DL,1952,N939AT,DSM,ATL,550,549.0,-1.0,15.0,604.0,131.0,141.0,110.0,743,854.0,16.0,901,910.0,9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Des Moines International Airport,Des Moines,IA,USA,41.53493,-93.66068,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
14,2024,12,17,4,WN,558,N263WN,CMH,BWI,550,548.0,-2.0,10.0,558.0,80.0,68.0,53.0,337,651.0,5.0,710,656.0,-14.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Port Columbus International Airport,Columbus,OH,USA,39.99799,-82.89188,Baltimore-Washington International Airport,Baltimore,MD,USA,39.17540,-76.66820
15,2024,12,17,4,WN,2360,N7720F,ABQ,MDW,550,557.0,7.0,16.0,613.0,165.0,153.0,132.0,1121,925.0,5.0,935,930.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919,Chicago Midway International Airport,Chicago,IL,USA,41.78598,-87.75242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819056,2024,12,17,4,WN,2692,N253WN,PIT,BNA,545,605.0,20.0,10.0,615.0,115.0,97.0,83.0,462,638.0,4.0,640,642.0,2.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Pittsburgh International Airport,Pittsburgh,PA,USA,40.49147,-80.23287,Nashville International Airport,Nashville,TN,USA,36.12448,-86.67818
5819057,2024,12,17,4,WN,2191,N220WN,RDU,BNA,545,542.0,-3.0,7.0,549.0,100.0,91.0,78.0,442,607.0,6.0,625,613.0,-12.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Raleigh-Durham International Airport,Raleigh,NC,USA,35.87764,-78.78747,Nashville International Airport,Nashville,TN,USA,36.12448,-86.67818
5819058,2024,12,17,4,WN,222,N373SW,SLC,LAX,545,548.0,3.0,35.0,623.0,120.0,125.0,81.0,590,644.0,9.0,645,653.0,8.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Salt Lake City International Airport,Salt Lake City,UT,USA,40.78839,-111.97777,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
5819071,2024,12,17,4,DL,2489,N332NB,PHL,DTW,545,543.0,-2.0,11.0,554.0,105.0,95.0,78.0,453,712.0,6.0,730,718.0,-12.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Philadelphia International Airport,Philadelphia,PA,USA,39.87195,-75.24114,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884


# Creating a copy of the filtered flights data

In [5]:
# This ensures that any modifications made to 'flights_needed_data' do not affect the original 'flights' DataFrame
flights_needed_data = flights.copy()

In [6]:
flights_needed_data.shape

(2137736, 45)

In [7]:
flights_needed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2137736 entries, 1 to 5819072
Data columns (total 45 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   YEAR                      int16  
 1   MONTH                     int8   
 2   DAY                       int8   
 3   DAY_OF_WEEK               int8   
 4   AIRLINE__CODE             object 
 5   FLIGHT_NUMBER             int16  
 6   TAIL_NUMBER               object 
 7   ORIGIN_AIRPORT_CODE       object 
 8   DESTINATION_AIRPORT_CODE  object 
 9   SCHEDULED_DEPARTURE       int16  
 10  DEPARTURE_TIME            float64
 11  DEPARTURE_DELAY           float64
 12  TAXI_OUT                  float64
 13  WHEELS_OFF                float64
 14  SCHEDULED_TIME            float64
 15  ELAPSED_TIME              float64
 16  AIR_TIME                  float64
 17  DISTANCE                  int16  
 18  WHEELS_ON                 float64
 19  TAXI_IN                   float64
 20  SCHEDULED_ARRIVAL         int

In [8]:
flights_needed_data.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLY_DATE,AIRLINE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE
1,2024,12,17,4,DL,1799,N603AT,CHA,ATL,549,552.0,3.0,15.0,607.0,56.0,48.0,25.0,106,632.0,8.0,645,640.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Chattanooga Metropolitan Airport (Lovell Field),Chattanooga,TN,USA,35.03527,-85.20379,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
6,2024,12,17,4,DL,1480,N319US,BDL,DTW,550,549.0,-1.0,20.0,609.0,120.0,129.0,104.0,549,753.0,5.0,750,758.0,8.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Bradley International Airport,Windsor Locks,CT,USA,41.93887,-72.68323,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884
7,2024,12,17,4,DL,1952,N939AT,DSM,ATL,550,549.0,-1.0,15.0,604.0,131.0,141.0,110.0,743,854.0,16.0,901,910.0,9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Des Moines International Airport,Des Moines,IA,USA,41.53493,-93.66068,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
14,2024,12,17,4,WN,558,N263WN,CMH,BWI,550,548.0,-2.0,10.0,558.0,80.0,68.0,53.0,337,651.0,5.0,710,656.0,-14.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Port Columbus International Airport,Columbus,OH,USA,39.99799,-82.89188,Baltimore-Washington International Airport,Baltimore,MD,USA,39.17540,-76.66820
15,2024,12,17,4,WN,2360,N7720F,ABQ,MDW,550,557.0,7.0,16.0,613.0,165.0,153.0,132.0,1121,925.0,5.0,935,930.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919,Chicago Midway International Airport,Chicago,IL,USA,41.78598,-87.75242


# Function to categorize scheduled arrival times into time segments

In [9]:
def categorize_time(SCHEDULED_ARRIVAL):
    # Categorize based on scheduled arrival time in 24-hour format
    if 500 <= SCHEDULED_ARRIVAL < 800:
        return 'Early morning'
    elif 800 <= SCHEDULED_ARRIVAL < 1100:
        return 'Late morning'
    elif 1100 <= SCHEDULED_ARRIVAL < 1400:
        return 'Around noon'
    elif 1400 <= SCHEDULED_ARRIVAL < 1700:
        return 'Afternoon'
    elif 1700 <= SCHEDULED_ARRIVAL < 2000:
        return 'Evening'
    elif 2000 <= SCHEDULED_ARRIVAL < 2300:
        return 'Night'
    elif SCHEDULED_ARRIVAL >= 2300 or SCHEDULED_ARRIVAL < 200:
        return 'Late night'
    elif 200 <= SCHEDULED_ARRIVAL < 500:
        return 'Dawn'

# Apply categorize_time function to the 'SCHEDULED_ARRIVAL' column to create 'ARRIVAL_TIME_SEGMENT'
flights_needed_data['ARRIVAL_TIME_SEGMENT'] = flights_needed_data['SCHEDULED_ARRIVAL'].apply(categorize_time)


In [10]:
flights_needed_data

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLY_DATE,AIRLINE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE,ARRIVAL_TIME_SEGMENT
1,2024,12,17,4,DL,1799,N603AT,CHA,ATL,549,552.0,3.0,15.0,607.0,56.0,48.0,25.0,106,632.0,8.0,645,640.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Chattanooga Metropolitan Airport (Lovell Field),Chattanooga,TN,USA,35.03527,-85.20379,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Early morning
6,2024,12,17,4,DL,1480,N319US,BDL,DTW,550,549.0,-1.0,20.0,609.0,120.0,129.0,104.0,549,753.0,5.0,750,758.0,8.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Bradley International Airport,Windsor Locks,CT,USA,41.93887,-72.68323,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,Early morning
7,2024,12,17,4,DL,1952,N939AT,DSM,ATL,550,549.0,-1.0,15.0,604.0,131.0,141.0,110.0,743,854.0,16.0,901,910.0,9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Des Moines International Airport,Des Moines,IA,USA,41.53493,-93.66068,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Late morning
14,2024,12,17,4,WN,558,N263WN,CMH,BWI,550,548.0,-2.0,10.0,558.0,80.0,68.0,53.0,337,651.0,5.0,710,656.0,-14.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Port Columbus International Airport,Columbus,OH,USA,39.99799,-82.89188,Baltimore-Washington International Airport,Baltimore,MD,USA,39.17540,-76.66820,Early morning
15,2024,12,17,4,WN,2360,N7720F,ABQ,MDW,550,557.0,7.0,16.0,613.0,165.0,153.0,132.0,1121,925.0,5.0,935,930.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919,Chicago Midway International Airport,Chicago,IL,USA,41.78598,-87.75242,Late morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819056,2024,12,17,4,WN,2692,N253WN,PIT,BNA,545,605.0,20.0,10.0,615.0,115.0,97.0,83.0,462,638.0,4.0,640,642.0,2.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Pittsburgh International Airport,Pittsburgh,PA,USA,40.49147,-80.23287,Nashville International Airport,Nashville,TN,USA,36.12448,-86.67818,Early morning
5819057,2024,12,17,4,WN,2191,N220WN,RDU,BNA,545,542.0,-3.0,7.0,549.0,100.0,91.0,78.0,442,607.0,6.0,625,613.0,-12.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Raleigh-Durham International Airport,Raleigh,NC,USA,35.87764,-78.78747,Nashville International Airport,Nashville,TN,USA,36.12448,-86.67818,Early morning
5819058,2024,12,17,4,WN,222,N373SW,SLC,LAX,545,548.0,3.0,35.0,623.0,120.0,125.0,81.0,590,644.0,9.0,645,653.0,8.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Southwest Airlines Co.,Salt Lake City International Airport,Salt Lake City,UT,USA,40.78839,-111.97777,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,Early morning
5819071,2024,12,17,4,DL,2489,N332NB,PHL,DTW,545,543.0,-2.0,11.0,554.0,105.0,95.0,78.0,453,712.0,6.0,730,718.0,-12.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-12-17,Delta Air Lines Inc.,Philadelphia International Airport,Philadelphia,PA,USA,39.87195,-75.24114,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,Early morning


In [11]:
flights['AIRLINE__CODE'].unique()

array(['DL', 'WN'], dtype=object)

In [12]:
flights_needed_data.value_counts('DIVERTED')

DIVERTED
0    2132545
1       5191
Name: count, dtype: int64

In [13]:
flights_needed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2137736 entries, 1 to 5819072
Data columns (total 46 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   YEAR                      int16  
 1   MONTH                     int8   
 2   DAY                       int8   
 3   DAY_OF_WEEK               int8   
 4   AIRLINE__CODE             object 
 5   FLIGHT_NUMBER             int16  
 6   TAIL_NUMBER               object 
 7   ORIGIN_AIRPORT_CODE       object 
 8   DESTINATION_AIRPORT_CODE  object 
 9   SCHEDULED_DEPARTURE       int16  
 10  DEPARTURE_TIME            float64
 11  DEPARTURE_DELAY           float64
 12  TAXI_OUT                  float64
 13  WHEELS_OFF                float64
 14  SCHEDULED_TIME            float64
 15  ELAPSED_TIME              float64
 16  AIR_TIME                  float64
 17  DISTANCE                  int16  
 18  WHEELS_ON                 float64
 19  TAXI_IN                   float64
 20  SCHEDULED_ARRIVAL         int

# Define columns by data type

In [14]:
numerical_cols = ['MONTH', 'DAY', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
                  'DISTANCE', 'SCHEDULED_ARRIVAL', 'DIVERTED', 'CANCELLED', 'AIR_SYSTEM_DELAY',
                  'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
categorical_cols = ['AIRLINE', 'ARRIVAL_TIME_SEGMENT']


# Define transformations for numerical columns: imputing and scaling

In [15]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Define transformations for categorical columns: imputing and one-hot encoding


In [16]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine both transformations in a ColumnTransformer


In [17]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Create a full pipeline; add your model at the end (e.g., DecisionTreeClassifier)

In [18]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MONTH', 'DAY',
                                                   'SCHEDULED_DEPARTURE',
                                                   'DEPARTURE_TIME',
                                                   'DEPARTURE_DELAY',
                                                   'DISTANCE',
                                                   'SCHEDULED_ARRIVAL',
                                                   'DIVERTED', 'CANCELLED',
                                                   'AIR_SYSTEM_DELAY',
                                                   'SECURITY_DELAY',
                                                   'AIRLINE_DELAY',
                                                   'LATE_AIRCRAFT_DELAY',
                                                   'WEATHER_DELAY']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['AIRLINE',
                                                   'ARRIVAL_TIME_SEGMENT'])])),
                ('classifier', DecisionTreeClassifier())])

# Creating the target column

In [19]:
result = []
for row in flights_needed_data['ARRIVAL_DELAY']:
  if row > 5:
    result.append(1)
  else:
    result.append(0) 

flights_needed_data['result'] = result
flights_needed_data.value_counts('result')

result
0    1595430
1     542306
Name: count, dtype: int64

In [20]:
test_data = flights_needed_data[flights_needed_data['MONTH'] >= 11] 
#test=test.drop(['FLY_DATE'], axis=1)
train_data = flights_needed_data[flights_needed_data['MONTH'] < 11]


In [ ]:
# Replace 'target_column_name' with the actual name of your target column
X_train = train_data.drop(columns=['result'])
y_train = train_data['result']

In [ ]:
# Replace 'target_column_name' with the actual name of your target column
X_test = test_data.drop(columns=['result'])
y_test = test_data['result']

In [ ]:
pipeline.fit(X_train,y_train)

In [ ]:
y_prob = pipeline.predict_proba(X_test)[:,1]

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
auc_score = roc_auc_score(y_test, y_pred)
auc_score

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
y_pred = pipeline.predict(flights_needed_data)

In [ ]:
flights['ACTUAL_DELAY'] = flights_needed_data['result']
flights

In [ ]:
flights['PREDICTED_DELAY'] = y_pred

In [ ]:
flights

In [ ]:
flights = flights.where(pd.notnull(flights), None)

In [ ]:
flights.isna().sum()

In [ ]:
chunk_size = 1000000
chunks = [flights[i:i + chunk_size] for i in range(0, len(flights), chunk_size)]

for chunk in chunks:
    # Ensure no NaN values remain by explicitly replacing NaN with None
    chunk = chunk.where(pd.notnull(chunk), None)
    
    # # Explicitly set the data type of each column to object
    # for col in chunk.columns:
    #     chunk[col] = chunk[col].astype(object)

    # Convert the DataFrame to a Snowflake-compatible DataFrame
    ins_train_sf = my_session.createDataFrame(
        chunk.values.tolist(),
        schema=chunk.columns.tolist()
    )
    
    # Write to Snowflake
    ins_train_sf.write.mode("append").save_as_table("TTH_DB.TTH_AIRLINE_SCHEMA.DELAY_CLASSIFIER_OUTPUT_2510")

In [ ]:
len(chunks)

In [ ]:
flights.info()

In [ ]:
my_session.sql('''
CREATE TABLE IF NOT EXISTS TTH_DB.TTH_AIRLINE_SCHEMA.DELAY_CLASSIFIER_OUTPUT_2510 (
    YEAR                      INT  NULL,
    MONTH                     INT NULL,
    DAY                       INT NULL,
    DAY_OF_WEEK               INT NULL,
    AIRLINE__CODE             STRING NULL,
    FLIGHT_NUMBER             INT NULL,
    TAIL_NUMBER               STRING NULL,
    ORIGIN_AIRPORT_CODE       STRING NULL,
    DESTINATION_AIRPORT_CODE  STRING NULL,
    SCHEDULED_DEPARTURE       INT NULL,
    DEPARTURE_TIME            FLOAT NULL,
    DEPARTURE_DELAY           FLOAT NULL,
    TAXI_OUT                  FLOAT NULL,
    WHEELS_OFF                FLOAT NULL,
    SCHEDULED_TIME            FLOAT NULL,
    ELAPSED_TIME              FLOAT NULL,
    AIR_TIME                  FLOAT NULL,
    DISTANCE                  INT NULL,
    WHEELS_ON                 FLOAT NULL,
    TAXI_IN                   FLOAT NULL,
    SCHEDULED_ARRIVAL         INT NULL,
    ARRIVAL_TIME              FLOAT NULL,
    ARRIVAL_DELAY             FLOAT NULL,
    DIVERTED                  INT NULL,
    CANCELLED                 INT NULL,
    CANCELLATION_REASON       STRING NULL,
    AIR_SYSTEM_DELAY          FLOAT NULL,
    SECURITY_DELAY            FLOAT NULL,
    AIRLINE_DELAY             FLOAT NULL,
    LATE_AIRCRAFT_DELAY       FLOAT NULL,
    WEATHER_DELAY             FLOAT NULL,
    FLY_DATE                  STRING NULL,
    AIRLINE                   STRING NULL,
    ORIGIN_AIRPORT            STRING NULL,
    ORIGIN_CITY               STRING NULL,
    ORIGIN_STATE              STRING NULL,
    ORIGIN_COUNTRY            STRING NULL,
    ORIGIN_LATITUDE           FLOAT NULL,
    ORIGIN_LONGITUDE          FLOAT NULL,
    DEST_AIRPORT              STRING NULL,
    DEST_CITY                 STRING NULL,
    DEST_STATE                STRING NULL,
    DEST_COUNTRY              STRING NULL,
    DEST_LATITUDE             FLOAT NULL,
    DEST_LONGITUDE            FLOAT NULL,
    ACTUAL_DELAY              INT NULL,
    PREDICTED_DELAY           INT NULL
);

''').collect()

In [ ]:
my_session.sql('drop table  TTH_DB.TTH_AIRLINE_SCHEMA.DELAY_CLASSIFIER_OUTPUT_2510;').collect()